Dataset : https://www.kaggle.com/c/fake-news/data#

###### Imorting Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##### Importing Dataset

In [7]:
df=pd.read_csv("fake_train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
#Dropping the nan values
df=df.dropna()

In [9]:
#Independent variable
X=df.iloc[:,:-1]

In [10]:
#Independent variable
y=df.iloc[:,-1]

In [11]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [12]:
X.shape,y.shape

((18285, 4), (18285,))

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [14]:
#Setting vocabulary size
voc_size=5000

##### One Hot Representation

In [15]:
messages=X.copy()

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\icons\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Dataset Preprocessing

In [33]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    #sustituting the values other than "a to z" and "A-Z" with a "space"
    review=re.sub("[^a-zA-Z]"," ",messages["title"][i])
    #converting every word in lowercase
    review=review.lower()
    review=review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words("english")]
    review= " ".join(review)
    corpus.append(review)

In [34]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [35]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3631, 3917, 3064, 185, 4433, 1721, 4132, 3157, 2462, 2464],
 [1080, 4620, 2243, 4141, 1149, 4430, 1687],
 [2399, 4901, 660, 1374],
 [3359, 2291, 580, 3487, 4059, 4045],
 [3569, 1149, 4816, 1174, 4711, 461, 1149, 991, 4841, 2683],
 [1236,
  4797,
  4666,
  1592,
  4784,
  282,
  4563,
  3954,
  386,
  4754,
  2902,
  640,
  1640,
  1015,
  1687],
 [4968, 722, 51, 106, 3829, 1615, 1942, 977, 4233, 143, 3089],
 [244, 4888, 4534, 4278, 1061, 2204, 282, 4145, 4233, 143, 3089],
 [3895, 2641, 4564, 2477, 2338, 2082, 2720, 4352, 282, 1293],
 [3553, 2304, 1245, 4878, 3446, 3257, 573, 4678],
 [3352, 4230, 4743, 1527, 2061, 3273, 1776, 3422, 1857, 1705, 2624],
 [3487, 3667, 4433, 2082, 282, 1061],
 [3855, 2845, 1021, 27, 1540, 3014, 692, 2886, 2093],
 [3823, 3995, 2152, 2438, 180, 2466, 2805, 4233, 143, 3089],
 [4669, 986, 1686, 3817, 3295, 4233, 143, 3089],
 [1572, 2570, 2447, 3433, 1580, 387, 539, 1198, 3893, 3651],
 [2998, 2803, 4620],
 [4077, 1966, 4858, 2574, 282, 750, 4727, 1687],
 [2369,

##### Embedding Representation

In [36]:
sent_length=20
#adding padding so that length of al the input sentences remain same
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3157 2462 2464]
 [   0    0    0 ... 1149 4430 1687]
 [   0    0    0 ... 4901  660 1374]
 ...
 [   0    0    0 ... 4233  143 3089]
 [   0    0    0 ... 1840  744 2139]
 [   0    0    0 ... 4258  939 2341]]


In [37]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3631,
       3917, 3064,  185, 4433, 1721, 4132, 3157, 2462, 2464])

##### Creating Model

In [45]:
#Unidirectional rnn
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
#Bidirectional rnn
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
len(embedded_docs),y.shape

(18285, (18285,))

In [41]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [42]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [47]:
#Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 19s 68ms/step - loss: 0.4458 - accuracy: 0.7666 - val_loss: 0.2029 - val_accuracy: 0.9160
Epoch 2/10
192/192 [==============================] - 11s 57ms/step - loss: 0.1322 - accuracy: 0.9512 - val_loss: 0.2093 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 11s 56ms/step - loss: 0.0875 - accuracy: 0.9692 - val_loss: 0.2472 - val_accuracy: 0.9125
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0556 - accuracy: 0.9817 - val_loss: 0.3585 - val_accuracy: 0.9117
Epoch 5/10
192/192 [==============================] - 12s 60ms/step - loss: 0.0349 - accuracy: 0.9889 - val_loss: 0.3443 - val_accuracy: 0.9135
Epoch 6/10
192/192 [==============================] - 11s 60ms/step - loss: 0.0213 - accuracy: 0.9940 - val_loss: 0.3906 - val_accuracy: 0.9079
Epoch 7/10
192/192 [==============================] - 11s 58ms/step - loss: 0.0196 - accuracy: 0.9953 - val_loss: 0.5055 - val_accuracy:

##### Performance Matrix and Accuracy

In [48]:
y_pred1=model1.predict_classes(X_test)

C:\Users\icons\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
#Building Confusion matrix
confusion_matrix(y_test,y_pred1)

array([[3152,  267],
       [ 312, 2304]], dtype=int64)

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.904059652029826

In [53]:
#Classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      3419
           1       0.90      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

